Step 1 - cleaning data from CSV file 

1.1 - deducting duplicte emails 

In [79]:

import pandas as pd

# Load the CSV
file_path = 'Data.csv'
# reading th csv file using pandas
df = pd.read_csv(file_path)

# Identify masked emails (those that contain "*****")
masked = df['email'].str.contains(r'\*{5}', na=False)
unmasked_emails = df[~masked]

# Count duplicates
total_unmasked = len(unmasked_emails)
unique_unmasked = unmasked_emails["email"].nunique()
duplicates_dropped = total_unmasked - unique_unmasked

print(f"Duplicate email entries dropped: {duplicates_dropped}")


Duplicate email entries dropped: 0


In [80]:
# if first_name is not present, create it from the name column and update the DataFrame
# Clean and normalize name column
# Clean and normalize name column
df['name'] = df['name'].astype(str).str.strip().str.replace(r'\s+', ' ', regex=True)

# Fill first_name where it's missing
mask_first_missing = df['first_name'].isnull()
df.loc[mask_first_missing, 'first_name'] = df.loc[mask_first_missing, 'name'].str.split().str[0]

# Fill last_name where it's missing and name has more than one word
mask_last_missing = df['last_name'].isnull() & (df['name'].str.split().str.len() > 1)
df.loc[mask_last_missing, 'last_name'] = df.loc[mask_last_missing, 'name'].str.split().str[-1]




In [81]:
# make Job Title - job_title
# name What is your LinkedIn profile? to linkedin_profile
df.rename(columns={
    'Job Title': 'job_title',
    'What is your LinkedIn profile?': 'linkedin_profile'
}, inplace=True)

In [82]:
df["has_joined_event"] = df["has_joined_event"].str.strip().str.lower().map({
    "yes": True,
    "no": False
})


# Print the only has joined event column with total number of rows
print(f"Total rows in 'has_joined_event' column: {len(df['has_joined_event'])}")
print(df["has_joined_event"].head(11))   



Total rows in 'has_joined_event' column: 648
0     False
1     False
2     False
3     False
4      True
5     False
6      True
7      True
8      True
9      True
10     True
Name: has_joined_event, dtype: bool


So, data gas been manpluted

1.3 dentify and flag rows where:
 - LinkedIn profile is missing or incomplete 
 - The job title is blank 

flaging linkdin profiles in 3 types
- valid 
- missing 
- incorrect 


In [83]:
# Clean and analyze the LinkedIn column

def check_linkedin_status(link):
    # Convert to string safely
    link = str(link).strip().lower()

    if link in ["", "nan", "none"]:
        return "Missing"
    elif "linkedin.com/in/" in link:
        return "Valid"
    elif "linkedin.com" in link:
        return "Incomplete"
    else:
        return "Missing"

    

df['linkdin_status'] = df['linkedin_profile'].apply(check_linkedin_status)

# Show results
df[['name', 'linkedin_profile', 'linkdin_status']].head(10), df['linkdin_status'].value_counts()


(                        name  \
 0                Venkatesh R   
 1                 Mark Jawut   
 2              Avinash Kumar   
 3                Wilson Juma   
 4                     Suneel   
 5  Aishwarya kumar choudhary   
 6            AAKASH JANGEETI   
 7               Arushi Yadav   
 8           Aashish Ailawadi   
 9             Aashish Sharma   
 
                                     linkedin_profile linkdin_status  
 0      https://linkedin.com/in/venkatesh-r-42845a282          Valid  
 1                                          MarkJawut        Missing  
 2                  linkedin.com/in/Avinash.kumar5167          Valid  
 3                      linkedin.com/in/wilsonjuma254          Valid  
 4                                                NaN        Missing  
 5  www.linkedin.com/in/ aishwarya-kumar-choudhary...          Valid  
 6     https://linkedin.com/in/aakash-royal-a8015821b          Valid  
 7  https://www.linkedin.com/in/arushi-yadav-68744...          Vali

doing similar for job title 

In [84]:


df['job_title'] = df['job_title'].astype(str).str.strip().str.split(',').str[0].str.strip()

import re

df['job_title_Status'] = df['job_title'].apply(
    lambda x: 'Missing'
    if pd.isna(x) or re.search(r'\b(none|null|na|n/a|none yet)\b', str(x).strip().lower())
    else 'Present'
)



# Count how many are missing vs present
job_title_status_counts = df['job_title_Status'].value_counts()

# Show a sample of the updated data
df[['name', 'job_title', 'job_title_Status']].head(10), job_title_status_counts 


#show details of missing job titles
missing_job_titles = df[df['job_title_Status'] == 'Missing'][['name', 'email', 'job_title']]    
print("Details of missing job titles:")
print(missing_job_titles.head())


Details of missing job titles:
                      name            email job_title
16      Okeowo Abdulraheem  *****@gmail.com  None yet
25   Abhishek Sanjay Dighe  *****@gmail.com      Null
160   Divya Somnath Shinde  *****@gmail.com        Na


Completing Step 1 by saving all the  saving all new data in a new csv file called cleaned_output.csv

In [85]:

# Saving to a CSV file
output_csv_path = 'cleaned_output.csv '
df.to_csv(output_csv_path, index=False)

problems i faced


In [86]:
# print all the job titles
print("All job titles:")
print(df['job_title'].unique())
# Print the first few rows of the cleaned DataFrame
print("First few rows of the cleaned DataFrame:")
print(df.head())


All job titles:
['Freelance' 'Student' 'Digital Marketing Specialist' 'Devops engineer'
 'Data Scientist' 'AI engineer' 'Unemployed' 'Sr. Software Engineer'
 'Data analyst' 'Associate Product Manager' 'Lead Engineer' 'Data Analyst'
 'Junior Software Engineer' 'Machine Learning Engineer' 'None yet'
 'Sr Product Manager' 'CTO' 'AI Enginer' 'Lead Product Manager'
 'Director Product Growth' 'VP Product & Tech' 'SPM' 'Null'
 'Product Marketer' 'TPM' 'Data Analyst Enthusiast' 'Information Analyst'
 'SDE' 'Aspiring data scientist' 'SE' 'Social media manager'
 'IX Developer' 'Student Data Analyst' 'Freelancer' 'Secretary' 'marketer'
 'ML Engineer' 'Developer' 'Product Engineer' 'pm' 'Director'
 'Technical Project Manager' 'AIML' 'Teaching Assitant'
 'Executive Director' 'YouTubers' 'Teacher' 'Strategy Manager' 'Dentist'
 'Data analytics' 'Analyst' 'Programmer' 'Product manager' 'Delivery Lead'
 'Data engineer' 'Drilling Fluids Engineer' 'Data Engineer' 'IT admin'
 'Kogo AI' 'Senior Product Man

In [87]:
#print linkedin profile of Abhishek Gupta
abhishek_gupta = df[df['name'].str.contains('Abhishek Gupta', case=False, na=False)]
print("LinkedIn profile of Abhishek Gupta:")
print(abhishek_gupta[['name', 'linkedin_profile']].head(1))


LinkedIn profile of Abhishek Gupta:
              name                             linkedin_profile
23  Abhishek Gupta  https://www.linkedin.com/in/abhishekguptas/
